In [ ]:
from IPython.display import clear_output, display, HTML, Image
import ipywidgets as widgets
import cv2
import numpy as np
import math

# Define global variables to keep track of the displayed images
displayed_image = None

def high_level(image, mask):
    Q = np.zeros_like(image)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            Q[i][j] = image[i][j] | mask
    return Q

def low_level(image, mask):
    Q = np.zeros_like(image)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            Q[i][j] = image[i][j] & mask
    return Q

def mid_level(image, mask1, mask2):
    Q = np.zeros_like(image)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            Q[i][j] = (image[i][j] & mask1)
            Q[i][j] = Q[i][j] | mask2
    return Q

gray_image = cv2.imread('./Nemo.PNG', cv2.IMREAD_GRAYSCALE)

# Check if the image was loaded successfully
if gray_image is not None:
    # Convert the image to a NumPy array
    image = np.array(gray_image)
    Quant = int(input("Enter Quantization size: "))
    if Quant not in [2,4,16,32,64,128,256] :
      print('Invalid Size');
    else :

      quantization_size = int(256 // Quant)
      k = int(math.log(quantization_size, 2))

      mask_str_low_level = '1' * (8 - k) + '0' * k
      Mask_low_level = int('11111111', 2) & int(mask_str_low_level, 2)

      Mask_high_level = int('1' * 8, 2) & ~Mask_low_level

      mask_str_middle_level = '1' * (8 - (k - 1)) + '0' * (k - 1)
      Mask_middle_level_1 = int('11111111', 2) & int(mask_str_middle_level, 2)
      Mask_middle_level_1 = int('1' * 8, 2) & ~Mask_middle_level_1

      # Save images from all three functions
      low_level_image = low_level(image, Mask_low_level)
      high_level_image = high_level(image, Mask_high_level)
      middle_level_image = mid_level(image, Mask_low_level, Mask_middle_level_1)

      cv2.imwrite("low_level_image.jpg", low_level_image)
      cv2.imwrite("high_level_image.jpg", high_level_image)
      cv2.imwrite("middle_level_image.jpg", middle_level_image)

      # Create button widgets for displaying images
      low_level_button = widgets.Button(description="Low Level")
      high_level_button = widgets.Button(description="High Level")
      middle_level_button = widgets.Button(description="Middle Level")

      # Function to display images when buttons are clicked
      def display_image(button, new_image):
          global displayed_image
          if displayed_image is not None:
              clear_output()
              display(HTML("<h2>Image Levels</h2>"))
              display(widgets.HBox([low_level_button, high_level_button, middle_level_button]))
          displayed_image = new_image
          display(Image(new_image))

      def display_low_level_image(button):
          low_level_image = low_level(image, Mask_low_level)
          display_image(button, "low_level_image.jpg")

      def display_high_level_image(button):
          high_level_image = high_level(image, Mask_high_level)
          display_image(button, "high_level_image.jpg")

      def display_middle_level_image(button):
          middle_level_image = mid_level(image, Mask_low_level, Mask_middle_level_1)
          display_image(button, "middle_level_image.jpg")

      # Attach functions to buttons
      low_level_button.on_click(display_low_level_image)
      high_level_button.on_click(display_high_level_image)
      middle_level_button.on_click(display_middle_level_image)

      # Display the buttons using HTML
      display(HTML("<h2>Image Levels</h2>"))
      display(widgets.HBox([low_level_button, high_level_button, middle_level_button]))
